<a href="https://colab.research.google.com/github/seven320/kaggle-base/blob/main/FP000_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Dec 29 07:44:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
from google.colab import output
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
!pip install transformers > /dev/null
!pip install einops
!pip install timm
!pip install git+https://github.com/albumentations-team/albumentations
output.clear()

In [5]:
import gc
import json
import math
import os
import random
import re
import time
import copy
import warnings
from contextlib import contextmanager

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GroupKFold, train_test_split, KFold
from sklearn.preprocessing import RobustScaler
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
# import cv2
# from torch.cuda.amp import autocast, GradScaler
# from einops import rearrange as Rearrange
from einops.layers.torch import Rearrange, Reduce
import timm

from tqdm.notebook import tqdm
from transformers import AdamW, get_cosine_schedule_with_warmup
from typing import Optional, Tuple
from xgboost import XGBRegressor

In [6]:
COMPE_NAME = ""
BASE_DIR = f"/content/drive/MyDrive/kaggle/{COMPE_NAME}"

NOTEBOOK_NAME = ""

In [ ]:
# INPUT_DIR = Path(os.path.join(BASE_DIR ,f"input"))

# SAVE_DIR = Path(os.path.join(BASE_DIR ,f"models/{NOTEBOOK_NAME}"))
# SAVE_DIR.mkdir(exist_ok=True, parents=True)

# OOF_DIR = Path(os.path.join(BASE_DIR, f"oof/{NOTEBOOK_NAME}"))
# OOF_DIR.mkdir(exist_ok=True, parents = True)

# SUB_DIR = Path(os.path.join(BASE_DIR, f"submission/{NOTEBOOK_NAME}"))
# SUB_DIR.mkdir(exist_ok=True, parents = True)

In [ ]:
train_df = pd.read_csv(os.path.join(INPUT_DIR, "train.csv"))
test_df = pd.read_csv(os.path.join(INPUT_DIR, "test.csv"))
sample_df = pd.read_csv(os.path.join(INPUT_DIR, "sample_submission.csv"))

In [ ]:
#############
## Utility ##
#############
@contextmanager
def timer(name: str):
    t0 = time.time()
    print(f"[{name}] start")
    yield
    print(f"[{name}] done - elapsed {time.time() - t0:.2f}s")

In [ ]:
class Config:
    COMPUTE_VAL = True
    DEBUG = False

    N_FOLD = 5
    TRAIN_FOLD = [0]
    SEED = 1234

    IMG_SIZE = 224
    
    EXP_NAME = NOTEBOOK_NAME
    N_EPOCHS = 100
    BS = 64
    LR = 1e-5  * BS
    WEIGHT_DECAY = 1e-3

    DEVICE = 'cuda'

    MODEL_NAME = 'efficientnet_b0'
    # FC_DIM = 512
    SCHEDULER_PARAMS = {
            "lr_start": 1e-5,
            "lr_max": 1e-5 * 32,
            "lr_min": 1e-6,
            "lr_ramp_ep": 5,
            "lr_sus_ep": 0,
            "lr_decay": 0.8,
        }

device = torch.device(Config.DEVICE)

In [ ]:
def seed_everything(seed):
    """
    Seeds basic parameters for reproductibility of results
    
    Arguments:
        seed {int} -- Number of the seed
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def set_seed(seed = Config.SEED):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [ ]:
def read_image(data_id, gray=False):
    file_path = data_id
    if gray:
        img = cv2.cvtColor(cv2.imread(file_path), cv2.COLOR_BGR2GRAY)
    else:
        img = cv2.cvtColor(cv2.imread(file_path), cv2.COLOR_BGR2RGB)
    return img

In [ ]:
class PetfinderDataset(Dataset):
  def __init__(self, df:pd.DataFrame, dir: str, aug_p: float = 0.0, transform = None, test = False):
      self.dfs = df
      self.aug_p = aug_p
      self.dir = dir
      self.transform = transform
      self.test = test

  def __len__(self):
      return len(self.dfs)

  def __getitem__(self, index:int):
    image = read_image(os.path.join(self.dir, self.dfs.loc[index, "Id"]+".jpg"))
    if self.transform:
        augmented = self.transform(image=image)
        image = augmented['image']
    
    if self.test:
        label = [-1]
    else:
        label = self.dfs.loc[index, "Pawpularity"] - 1

    d = {
        "X": image,
        "y" : torch.tensor(label).float(),
    }

    return d